In [140]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io
from datetime import datetime
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
file_loc = "../Matlab/EERdatabase.xlsx"

In [111]:
# basically after they read in the file they do some calculation and turn them
# into a file call data_vars.mat
# 

# replicates the reading phase

# read in the files
cpi = pd.read_excel(file_loc, sheet_name="cpi",engine="openpyxl") 

USD = pd.read_excel(file_loc, sheet_name="ner_eop",engine="openpyxl") 

gdp = pd.read_excel(file_loc, sheet_name="gdppc_ppp",engine="openpyxl") 

nfa = pd.read_excel(file_loc, sheet_name="nfa2gdp",engine="openpyxl") 

tot = pd.read_excel(file_loc, sheet_name="tot_all",engine="openpyxl") 

weights = pd.read_excel(file_loc, sheet_name="weights_static",engine="openpyxl") 

# pre cleaning / define some variables that would be useful
time_line = cpi.iloc[:,0]
time_line = pd.DataFrame(time_line)
time_line.columns = ["Time"]
time_line["Time"] = pd.PeriodIndex(time_line["Time"], freq='Q').to_timestamp()
time_line["Year"] = time_line["Time"].dt.year
time_line["Q"] = time_line["Time"].dt.quarter

countries = list(cpi.columns)[1:]

# the first columns is always time, we DO NOT need them in the calculation
cpi = cpi.iloc[:,1:].to_numpy()
gdp = gdp.iloc[:,1:].to_numpy()
USD = USD.iloc[:,1:].to_numpy()
nfa = nfa.iloc[:,1:].to_numpy()
tot = tot.iloc[:,1:].to_numpy()

# weights

# following their reindexing into alphabetic order
weights.columns = ["country", 
"time", 
"USA", 
"GBR", 
"DNK",
"NOR",
"SWE", 
"CHE",
"CAN",
"JPN",
"EA",
"AUS",
"NZL"]
weights.set_index("country", inplace=True)
weights = weights.reindex(index=sorted(weights.index), columns=sorted(weights.columns))
# they fill the na with 0 for weights
weights = weights.fillna(0)
weights.drop(columns="time", inplace=True)
weights = weights.to_numpy()


# calculations below 

# log of relative CPI
rpi = np.log(cpi)- np.log(cpi)@weights
#  log of USD rate (increase stands for appreciation of dom. currency)
ner = np.log(USD)@weights - np.log(USD)
# log of USD real rate (increase stands for appreciation)
rer = ner + rpi

# log of relative gdp per capita
rgdp = np.log(gdp) - np.log(gdp)@weights

# log of relative terms of trade

rtot = np.log(tot) - np.log(tot)@weights

# relative nfa (choose which one)
rnfa = nfa-nfa@weights


### in the original code they get rid of denmark 
rpi = np.delete(rpi ,3,1)  
ner = np.delete(ner,3,1)  
rer = np.delete(rer,3,1) 
rgdp = np.delete(rgdp,3,1) 
rtot = np.delete(rtot,3,1) 
rnfa = np.delete(rnfa, 3,1)
countries.remove("DNK")


In [120]:
# A2_EqRERstimates

# periods in the paper (-1 in the end because Python)
period_start = 4*(1994-1975) + (4-1) +1 - 1
period_end = time_line.shape[0]

# this is the recursive part
for i in range(period_start, period_end):
    rerR = rer[0:i,:]
    gdpR = rgdp[0:i,:]
    nfaR = rnfa[0:i,:]
    totR = rtot[0:i,:]

79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175


In [173]:
# the regression table is the last recursive, meaning the from 80 to 176
# because its Python the structure below makes way more sense
timeR = time_line.iloc[:period_end,:]

rerR = pd.DataFrame(rer[0:period_end,:], columns=countries)
gdpR = pd.DataFrame(rgdp[0:period_end,:], columns=countries)
nfaR = pd.DataFrame(rnfa[0:period_end,:], columns=countries)
totR = pd.DataFrame(rtot[0:period_end,:], columns=countries)
 
rerR = pd.concat([timeR["Time"], rerR], axis=1)
gdpR =  pd.concat([timeR["Time"], gdpR], axis=1)
nfaR =  pd.concat([timeR["Time"], nfaR], axis=1)
totR =  pd.concat([timeR["Time"], totR], axis=1)

rerR = rerR.melt(id_vars="Time", var_name="Country", value_name="rerR")
gdpR = gdpR.melt(id_vars="Time", var_name="Country", value_name="gdpR")
nfaR = nfaR.melt(id_vars="Time", var_name="Country", value_name="nfaR")
totR = totR.melt(id_vars="Time", var_name="Country", value_name="totR")


out = pd.merge(rerR, gdpR, how="left", left_on=["Time", "Country"], right_on=["Time", "Country"])
out = pd.merge(out, nfaR, how="left", left_on=["Time", "Country"], right_on=["Time", "Country"])
out = pd.merge(out, totR, how="left", left_on=["Time", "Country"], right_on=["Time", "Country"])

out["Country"] = out["Country"].astype("category")
out.set_index(['Country', 'Time'], inplace=True)
exog_vars = out[["gdpR", "nfaR", "totR"]]
mod = PanelOLS(out.rerR, exog_vars, entity_effects=True, time_effects=True)
# the fixed effects of both the country and time_effects is closest to the results
fe_te_res = mod.fit()

C:\Anaconda3\lib\site-packages\linearmodels\panel\data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):


In [174]:
fe_te_res

Dep. Variable:,rerR,R-squared:,0.2449
Estimator:,PanelOLS,R-squared (Between):,-0.0293
No. Observations:,1760,R-squared (Within):,0.2436
Date:,"Tue, Feb 08 2022",R-squared (Overall):,-0.0280
Time:,20:00:52,Log-likelihood,1497.7
Cov. Estimator:,Unadjusted,,
,,F-statistic:,169.99
Entities:,10,P-value,0.0000
Avg Obs:,176.00,Distribution:,"F(3,1572)"
Min Obs:,176.00,,
Max Obs:,176.00,F-statistic (robust):,169.99


In [139]:
X = np.hstack((gdpR,nfaR, totR))
Y = rerR.copy()
X_vec = np.hstack((gdpR.flatten(), nfaR.flatten(), totR.flatten()))

In [135]:
mod = sm.OLS(Y,X).fit()

In [141]:
from linearmodels.datasets import wage_panel

data = wage_panel.load()
year = pd.Categorical(data.year)

In [142]:
data

,nr,year,black,exper,hisp,hours,married,educ,union,lwage,expersq,occupation
0,13,1980,0,1,0,2672,0,14,0,1.197540,1,9
1,13,1981,0,2,0,2320,0,14,1,1.853060,4,9
2,13,1982,0,3,0,2940,0,14,0,1.344462,9,9
3,13,1983,0,4,0,2960,0,14,0,1.433213,16,9
4,13,1984,0,5,0,3071,0,14,0,1.568125,25,5
...,...,...,...,...,...,...,...,...,...,...,...,...
4355,12548,1983,0,8,0,2080,1,9,0,1.591879,64,5
4356,12548,1984,0,9,0,2080,1,9,1,1.212543,81,5
4357,12548,1985,0,10,0,2080,1,9,0,1.765962,100,5
4358,12548,1986,0,11,0,2080,1,9,1,1.745894,121,5
